In [7]:
!pip install pdfplumber -q
!pip install PyPDF2 -q
!pip install python-Levenshtein -q
!pip install scikit-learn -q

In [8]:
import re
import json
import hashlib
from typing import List, Dict, Any, Optional
from dataclasses import dataclass
from datetime import datetime
import unicodedata
from difflib import SequenceMatcher
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from PyPDF2 import PdfReader
import pdfplumber

In [ ]:
@dataclass
class PageData:
    page: int
    raw_text: str
    normalized_text: str
    chapter: Optional[str]
    article: Optional[str]
    tables: List[List[str]]
    clean_text: str

@dataclass
class DocumentStructure:
    doc_id: str
    nome_doc: str
    versao: str
    data_publicacao: str
    pagina_inicial: int
    pagina_final: int
    paginas: List[PageData]

class CompleteDocumentProcessor:
    def __init__(self):
        self.siglas = {
            'TCC': 'Trabalho de Conclusão de Curso',
            'BSI': 'Bacharelado em Sistemas de Informação',
            'ACC': 'Atividades Complementares de Currículo',
            'AC': 'Atividades Complementares',
            'PPC': 'Projeto Pedagógico do Curso',
            'CH': 'Carga Horária',
            'MEC': 'Ministério da Educação',
            'SIGAA': 'Sistema Integrado de Gestão de Atividades Acadêmicas',
            'CEPE': 'Conselho de Ensino, Pesquisa e Extensão',
            'CNE': 'Conselho Nacional de Educação',
            'LDB': 'Lei de Diretrizes e Bases da Educação Nacional',
            'IES': 'Instituição de Ensino Superior'
        }
        self.similarity_threshold = 0.85

    def extract_complete_text_with_tables(self, pdf_path: str) -> List[Dict[str, Any]]:
        """
        Extrai texto e tabelas
        """
        pages_data = []

        with pdfplumber.open(pdf_path) as pdf:
            total_pages = len(pdf.pages)
            print(f"Processando {total_pages} páginas...")

            for page_num, page in enumerate(pdf.pages, 1):

                # texto
                text = page.extract_text() or ""
                raw_text = " ".join(text.split())  # limpeza básica

                # tabelas
                tables = []
                page_tables = page.extract_tables()
                if page_tables:
                    for table in page_tables:
                        if table and any(any(cell for cell in row if cell) for row in table):
                            cleaned_table = []
                            for row in table:
                                cleaned_row = [str(cell).strip() if cell else "" for cell in row]
                                if any(cleaned_row):  #
                                    cleaned_table.append(cleaned_row)
                            if cleaned_table:
                                tables.append(cleaned_table)

                # detectar capítulo e artigo na página
                chapter, article = self._detect_structure_on_page(raw_text)

                # normalizar texto
                normalized_text = self.preprocess_text(raw_text)

                # texto limpo
                clean_text = self.clean_text(raw_text)

                pages_data.append({
                    "page": page_num,
                    "raw_text": raw_text,
                    "normalized_text": normalized_text,
                    "chapter": chapter,
                    "article": article,
                    "tables": tables,
                    "clean_text": clean_text
                })

                if page_num % 10 == 0:
                    print(f"     Páginas processadas: {page_num}/{total_pages}")

        print(f"     Extração concluída: {len(pages_data)} páginas")
        return pages_data

    def _detect_structure_on_page(self, text: str) -> tuple[Optional[str], Optional[str]]:
        """
        Detecta capítulo e artigo em uma página
        """
        chapter = None
        article = None

        lines = text.split('\n')

        for line in lines:
            line = line.strip()

            # detectar capítulo
            chapter_match = re.search(
                r'(CAP[ÍI]TULO|T[ÍI]TULO)\s+([IVXLCDM]+|\d+(\.\d+)*)',
                line,
                re.IGNORECASE
            )
            if chapter_match:
                chapter = line
                continue

            # detectar padrões numéricos como "6.5", "3.2.1"
            numeric_chapter = re.search(r'^\d+(\.\d+)+', line)
            if numeric_chapter and not chapter:
                chapter = line
                continue

            # detectar artigo
            artigo_match = re.search(r'Art\.?\s*\d+[º°]?\.?', line)
            if artigo_match:
                article = artigo_match.group(0)
                break

        return chapter, article

    def preprocess_text(self, text: str) -> str:
        """
        Normalização do texto para deduplicação:
        - Minúsculas, remoção de pontuação, etc.
        """
        if not text:
            return ""

        # converter para minúsculas
        text = text.lower()

        # expandir siglas
        for sigla, expansao in self.siglas.items():
            text = re.sub(rf'\b{sigla}\b', expansao.lower(), text, flags=re.IGNORECASE)

        # normalizar caracteres (remover acentos)
        text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')

        # remover pontuação e caracteres especiais
        text = re.sub(r'[^\w\s]', ' ', text)

        # padronizar formato de artigos e parágrafos
        text = re.sub(r'artigo\s+(\d+)', r'art \1', text)
        text = re.sub(r'art\.?\s*(\d+)', r'art \1', text)
        text = re.sub(r'parágrafo\s+único', 'paragrafo unico', text)
        text = re.sub(r'§\s*(\d+)', r'paragrafo \1', text)

        # remover espaços extras e normalizar
        text = re.sub(r'\s+', ' ', text).strip()

        return text

    def clean_text(self, text: str) -> str:
        """
        Limpeza básica do texto para deduplicação
        """
        if not text:
            return ""

        # remover espaços extras e normalizar quebras
        text = re.sub(r'\s+', ' ', text).strip()
        return text

    def deduplicate_pages(self, pages_data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """
        Remove páginas duplicadas ou muito similares
        """

        if len(pages_data) <= 1:
            return pages_data

        # deduplicação exata
        unique_pages = self._remove_exact_duplicates(pages_data)
        print(f"  Fase 1 - Duplicatas exatas: {len(pages_data) - len(unique_pages)} removidas")

        # deduplicação por similaridade
        final_pages = self._remove_similar_pages(unique_pages)
        print(f"  Fase 2 - Páginas similares: {len(unique_pages) - len(final_pages)} removidas")

        return final_pages

    def _remove_exact_duplicates(self, pages_data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """Remove páginas com conteúdo exatamente igual"""
        unique_pages = []
        seen_hashes = set()

        for page in pages_data:
            # criar hash do conteúdo normalizado
            content_hash = hashlib.md5(page['normalized_text'].encode()).hexdigest()

            if content_hash not in seen_hashes:
                seen_hashes.add(content_hash)
                unique_pages.append(page)
            else:
                print(f"    🗑️ Removendo página {page['page']} (duplicata exata)")

        return unique_pages

    def _remove_similar_pages(self, pages_data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """Remove páginas com conteúdo muito similar"""
        if len(pages_data) <= 1:
            return pages_data

        # extrair textos para comparação
        texts = [page['normalized_text'] for page in pages_data]

        # usar TF-IDF e similaridade de cosseno
        vectorizer = TfidfVectorizer(min_df=1, max_df=0.9)
        try:
            tfidf_matrix = vectorizer.fit_transform(texts)
            cosine_sim = cosine_similarity(tfidf_matrix)

            to_keep = set(range(len(pages_data)))

            for i in range(len(pages_data)):
                if i in to_keep:
                    for j in range(i + 1, len(pages_data)):
                        if j in to_keep and cosine_sim[i][j] >= self.similarity_threshold:
                            to_keep.remove(j)
                            print(f"  Removendo página {pages_data[j]['page']} "
                                  f"(similar à página {pages_data[i]['page']})")

            return [pages_data[i] for i in sorted(to_keep)]

        except Exception as e:
            print(f"  Erro na deduplicação fuzzy: {e}")
            return pages_data  # retorna todas se houver erro

    def process_complete_document(self, pdf_path: str, metadata: Dict[str, str]) -> DocumentStructure:
        """Processa documento  com todas as páginas no formato solicitado"""
        print(f"PROCESSANDO DOCUMENTO : {metadata['nome_doc']}")


        print("Extraindo todas as páginas com texto e tabelas")
        pages_data = self.extract_complete_text_with_tables(pdf_path)

        if not pages_data:
            raise ValueError(f"Não foi possível extrair texto do documento: {metadata['nome_doc']}")

        # deduplicação
        final_pages = self.deduplicate_pages(pages_data)

        # converter para objetos PageData
        paginas_objects = []
        for page in final_pages:
            paginas_objects.append(PageData(
                page=page['page'],
                raw_text=page['raw_text'],
                normalized_text=page['normalized_text'],
                chapter=page['chapter'],
                article=page['article'],
                tables=page['tables'],
                clean_text=page['clean_text']
            ))

        # criar estrutura final do documento
        document = DocumentStructure(
            doc_id=metadata['doc_id'],
            nome_doc=metadata['nome_doc'],
            versao=metadata['versao'],
            data_publicacao=metadata['data_publicacao'],
            pagina_inicial=1,
            pagina_final=len(pages_data),  # numero total de páginas originais
            paginas=paginas_objects
        )

        print(f"Processamento concluído: {len(final_pages)} páginas únicas")
        return document

def save_to_jsonl(documents: List[DocumentStructure], output_path: str):
    """Salva documentos no formato JSONL exato solicitado"""
    with open(output_path, 'w', encoding='utf-8') as f:
        for doc in documents:

            doc_dict = {
                "doc_id": doc.doc_id,
                "nome_doc": doc.nome_doc,
                "versao": doc.versao,
                "data_publicacao": doc.data_publicacao,
                "pagina_inicial": doc.pagina_inicial,
                "pagina_final": doc.pagina_final,
                "paginas": [
                    {
                        "page": page.page,
                        "raw_text": page.raw_text,
                        "normalized_text": page.normalized_text,
                        "chapter": page.chapter,
                        "article": page.article,
                        "tables": page.tables,
                        "clean_text": page.clean_text
                    }
                    for page in doc.paginas
                ]
            }

            f.write(json.dumps(doc_dict, ensure_ascii=False) + '\n')

def generate_report(documents: List[DocumentStructure]):
    """Gera relatório  dos documentos processados"""
    print("\n" + "="*70)
    print("="*70)

    total_paginas = 0
    total_tabelas = 0

    for doc in documents:
        doc_paginas = len(doc.paginas)
        doc_tabelas = sum(len(page.tables) for page in doc.paginas)
        total_paginas += doc_paginas
        total_tabelas += doc_tabelas

        print(f"\n {doc.nome_doc}")
        print(f"    Páginas processadas: {doc_paginas}")
        print(f"    Tabelas extraídas: {doc_tabelas}")
        print(f"    ID: {doc.doc_id}")
        print(f"    Versão: {doc.versao} - {doc.data_publicacao}")

    print(f"\n TOTAL GERAL:")
    print(f"    Documentos: {len(documents)}")
    print(f"    Páginas: {total_paginas}")
    print(f"    Tabelas: {total_tabelas}")

def process_all_documents():
    """Processa os documentos"""
    processor = CompleteDocumentProcessor()
    documents = []

    documentos_config = [
        {
            'path': '/content/sample_data/regulamento_tcc.pdf',
            'metadata': {
                'doc_id': 'TCC_BSI_2024_001',
                'nome_doc': 'REGULAMENTO TCC BSI',
                'versao': '1.0',
                'data_publicacao': '2024-01-15'
            }
        },
        {
            'path': '/content/sample_data/estatuto_2025.pdf',
            'metadata': {
                'doc_id': 'ESTATUTO_2025_001',
                'nome_doc': 'Estatuto - Setembro de 2025',
                'versao': '2.0',
                'data_publicacao': '2025-09-01'
            }
        },
        {
            'path': '/content/sample_data/regulamento_atividades_complementares.pdf',
            'metadata': {
                'doc_id': 'ACC_2024_001',
                'nome_doc': 'REGULAMENTO DAS ATIVIDADES COMPLEMENTARES',
                'versao': '1.1',
                'data_publicacao': '2024-03-20'
            }
        }
    ]


    print("="*60)

    for doc_config in documentos_config:
        try:
            print(f"\n Processando: {doc_config['metadata']['nome_doc']}")
            document = processor.process_complete_document(
                doc_config['path'],
                doc_config['metadata']
            )
            documents.append(document)
            print(f"Concluído: {doc_config['metadata']['nome_doc']}")
        except Exception as e:
            print(f"Erro ao processar {doc_config['metadata']['nome_doc']}: {e}")

    # salvar resultados
    if documents:

        output_file = f"output.jsonl"

        save_to_jsonl(documents, output_file)

        # gerar relatório
        generate_report(documents)

        print(f"\nArquivo salvo: {output_file}")

    return documents

if __name__ == "__main__":
    documents = process_all_documents()


 Processando: REGULAMENTO TCC BSI
PROCESSANDO DOCUMENTO COMPLETO: REGULAMENTO TCC BSI
Extraindo todas as páginas com texto e tabelas
Processando 13 páginas...
     Páginas processadas: 10/13
     Extração concluída: 13 páginas
  Fase 1 - Duplicatas exatas: 0 removidas
  Removendo página 12 (similar à página 11)
  Fase 2 - Páginas similares: 1 removidas
Processamento concluído: 12 páginas únicas
Concluído: REGULAMENTO TCC BSI

 Processando: Estatuto - Setembro de 2025
PROCESSANDO DOCUMENTO COMPLETO: Estatuto - Setembro de 2025
Extraindo todas as páginas com texto e tabelas
Processando 54 páginas...
     Páginas processadas: 10/54
     Páginas processadas: 20/54
     Páginas processadas: 30/54
     Páginas processadas: 40/54
     Páginas processadas: 50/54
     Extração concluída: 54 páginas
  Fase 1 - Duplicatas exatas: 0 removidas
  Fase 2 - Páginas similares: 0 removidas
Processamento concluído: 54 páginas únicas
Concluído: Estatuto - Setembro de 2025

 Processando: REGULAMENTO DAS A